<a href="https://colab.research.google.com/github/abolfazlshahsavaryyy/DT/blob/main/DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [82]:
def compute_gini(*p):
    if not (0.999 <= sum(p) <= 1.001):
        raise ValueError("The probabilities must sum to 1.")

    result = 1
    for i in p:
        result -= i**2

    return result

In [83]:
def compute_entropy(*p):
    if not (0.999 <= sum(p) <= 1.001):
        raise ValueError("The probabilities must sum to 1.")

    result = 0
    for i in p:
        if i > 0:
            result += i * np.log2(i)

    return -result

In [84]:
def load_dataset(url:str)->pd.DataFrame:
    raw_df=pd.read_csv(url)
    return raw_df


In [85]:
def dropnan_df(raw_df:pd.DataFrame)->pd.DataFrame:

    return raw_df.dropna()


In [86]:
def count_values_columns(df:pd.DataFrame,cols:list):
    for s in cols:
        print(df[s].value_counts())


In [87]:
def Classify_continuous(df:pd.DataFrame,col_name ):

    col=df[col_name]
    min=np.min(col)
    max=np.max(col)
    mean=np.mean([max,min])
    mean_75=np.mean([max,mean])
    mean_25=np.mean([min,mean])
    df.loc[df[col_name]<=min,col_name]=0
    df.loc[(df[col_name]>min )& (df[col_name]<=mean_25),col_name]=1
    df.loc[(df[col_name]>mean_25) & (df[col_name]<=mean),col_name]=2
    df.loc[(df[col_name]>mean) & (df[col_name]<=mean_75),col_name]=3
    df.loc[(df[col_name]>mean_75) & (df[col_name]<=max),col_name]=4
    df.loc[df[col_name]>max ,col_name]=5


In [88]:
def custome_classify_age(df:pd.DataFrame,col="Age")->pd.DataFrame:
    df.loc[(df[col]>=1 )&(df[col]<=6),col]=0
    df.loc[(df[col]==7 ),col]=1
    df.loc[(df[col]==8 ),col]=2
    df.loc[(df[col]==9 ),col]=3
    df.loc[(df[col]==10 ),col]=4
    df.loc[(df[col]==11 ),col]=5
    #over 60 years
    df.loc[(df[col]>=12 )&(df[col]<=13),col]=6



In [89]:
def custome_classify_bmi(df:pd.DataFrame,col="BMI"):
    df.loc[(df[col]<=18.5),col]=0
    df.loc[(df[col]>18.5)&(df[col]<=25),col]=1
    df.loc[(df[col]>25)&(df[col]<=30),col]=2
    df.loc[(df[col]>30)&(df[col]<=35),col]=3
    df.loc[(df[col]>35)&(df[col]<=40),col]=4
    df.loc[(df[col]>40),col]=5
    return df

In [90]:
def custome_classify_PhysHlth_MentHlth(df:pd.DataFrame):
    df.loc[(df["MentHlth"]<6),"MentHlth"]=0
    df.loc[(df["MentHlth"]>=6),"MentHlth"]=1

    df.loc[(df["PhysHlth"]<6),"PhysHlth"]=0
    df.loc[(df["PhysHlth"]>=6),"PhysHlth"]=1



In [91]:
def map_df(df:pd.DataFrame)-> pd.DataFrame:
    gender_map={
        "Female":0,
        "Male":1
    }
    YesOrNo={
        "No":0,
        "Yes":1
    }
    df["Sex"]=df["Sex"].map(gender_map)
    df["HighChol"]=df["HighChol"].map(YesOrNo)
    df["CholCheck"]=df["CholCheck"].map(YesOrNo)
    df["Smoker"]=df["Smoker"].map(YesOrNo)
    df["HeartDiseaseorAttack"]=df["HeartDiseaseorAttack"].map(YesOrNo)
    df["PhysActivity"]=df["PhysActivity"].map(YesOrNo)
    df["Fruits"]=df["Fruits"].map(YesOrNo)
    df["Veggies"]=df["Veggies"].map(YesOrNo)
    df["HvyAlcoholConsump"]=df["HvyAlcoholConsump"].map(YesOrNo)
    df["DiffWalk"]=df["DiffWalk"].map(YesOrNo)
    df["Stroke"]=df["Stroke"].map(YesOrNo)
    df["HighBP"]=df["HighBP"].map(YesOrNo)
    df["Diabetes"]=df["Diabetes"].map(YesOrNo)


    return df

In [92]:
def split_train_test_valid(df, random_state=42, test_size=0.2, valid_size=0.2):
    n = len(df)
    np.random.seed(random_state)
    indexes = np.random.permutation(n)


    test_size_df = int(n * test_size)
    valid_size_df = int((n - test_size_df) * valid_size)
    train_size_df = n - (test_size_df + valid_size_df)


    df_train = df.iloc[indexes[:train_size_df]].reset_index(drop=True)
    df_test = df.iloc[indexes[train_size_df:train_size_df + test_size_df]].reset_index(drop=True)
    df_valid = df.iloc[indexes[train_size_df + test_size_df:]].reset_index(drop=True)

    return df_train, df_test, df_valid

In [93]:
def split_X_y(df,y_label="class"):
    y=df[y_label]
    X=df.drop(columns=[y_label])
    return X,y

In [96]:
url_dataset="https://raw.githubusercontent.com/abolfazlshahsavaryyy/DT/main/Dataset.csv"
df=load_dataset(url_dataset)
df=dropnan_df(df)
map_df(df)
custome_classify_age(df,"Age")
custome_classify_bmi(df,"BMI")
custome_classify_PhysHlth_MentHlth(df)
train_df,test_df,valid_df=split_train_test_valid(df,random_state=42,test_size=0.2,valid_size=0.2)
X_train,y_train=split_X_y(train_df,"Diabetes")
X_test,y_test=split_X_y(test_df,"Diabetes")
X_valid,y_valid=split_X_y(valid_df,"Diabetes")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   70692 non-null  float64
 1   Sex                   70692 non-null  int64  
 2   HighChol              70692 non-null  int64  
 3   CholCheck             70692 non-null  int64  
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  int64  
 6   HeartDiseaseorAttack  70692 non-null  int64  
 7   PhysActivity          70692 non-null  int64  
 8   Fruits                70692 non-null  int64  
 9   Veggies               70692 non-null  int64  
 10  HvyAlcoholConsump     70692 non-null  int64  
 11  GenHlth               70692 non-null  float64
 12  MentHlth              70692 non-null  float64
 13  PhysHlth              70692 non-null  float64
 14  DiffWalk              70692 non-null  int64  
 15  Stroke             